In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from keras.models import load_model

In [3]:
#import CSV to be read as a pandas dataframe
data = pd.read_csv('/content/drive/MyDrive/Preprocessed_BRCA_RNA_data_normalized.csv', header = 0)



#encode categorical variables to numerical
labelenc = LabelEncoder()
data['Class'] = labelenc.fit_transform(data['Class'])
data['Sample-id'] = labelenc.fit_transform(data['Sample-id'])

print(data)

#get labels from original data and use encoded data as x
x = data
y = data['Class']

print(x.shape)
print(y.shape)

      Sample-id  Class  ?|100130426  ...  ZZZ3|26009  psiTPTE22|387590  tAKR|389932
0             0      2       0.0000  ...   1180.4565            1.7233       0.0000
1             1      2       0.0000  ...    406.7428          926.5905       0.0000
2             2      2       0.9066  ...    509.5195           35.3581       0.0000
3             3      2       0.0000  ...    700.8688           66.6115       0.0000
4             4      2       0.0000  ...    881.7021          187.2340       0.0000
...         ...    ...          ...  ...         ...               ...          ...
1090       1090      2       0.0000  ...    443.0869          724.9033       0.0000
1091       1091      2       0.0000  ...    670.0787           98.4252       0.0000
1092       1092      2       0.0000  ...    953.7081          235.2385       0.9447
1093       1093      2       0.0000  ...    750.8288          238.9272       0.0000
1094       1094      2       0.0000  ...    462.1140           20.8786      

In [4]:
#divide into train and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(876, 20533)
(219, 20533)
(876,)
(219,)


In [5]:
#scale data & normalize
t = MinMaxScaler()
t.fit(x_train)
t.fit(x_test)

x_train = t.transform(x_train)
x_test = t.transform(x_test)

In [6]:
#set classifier to vote of 3 neighbors, weighted by distance from k
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')


In [7]:
#Gets validations stats iterated from every fold 
pred_dict = []
def cscorer(clf, x, y):
  y_pred = clf.predict(x)
  metric = metrics.classification_report(y, y_pred)
  pred_dict.append(metric)
  return 1

all_pred = cross_val_score(knn, x, y, cv=5, scoring= cscorer)


for i in pred_dict:
  print('5-fold cross validation accuracies are: ')
  print(i)

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.85      1.00      0.92        22
           1       0.65      0.57      0.60        23
           2       0.95      0.94      0.95       174

    accuracy                           0.91       219
   macro avg       0.81      0.84      0.82       219
weighted avg       0.91      0.91      0.91       219

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.91      0.95      0.93        22
           1       0.73      0.70      0.71        23
           2       0.95      0.95      0.95       174

    accuracy                           0.93       219
   macro avg       0.86      0.87      0.87       219
weighted avg       0.93      0.93      0.93       219

5-fold cross validation accuracies are: 
              precision    recall  f1-score   support

           0       0.79      1.00      0.88        22
      